In [1]:
import pandas
from json import loads
from tqdm import tqdm
from utilities import *

In [2]:
train_data = pandas.read_parquet("invoices-and-receipts_ocr/train-invoices-and-receipts_ocr_v1.parquet")
#valid_data = pandas.read_parquet("invoices-and-receipts_ocr/valid-invoices-and-receipts_ocr_v1.parquet")
#test_data = pandas.read_parquet("invoices-and-receipts_ocr/test-invoices-and-receipts_ocr_v1.parquet")

In [3]:
train_data['parsed_data'] = train_data['parsed_data'].astype('string')
train_data['raw_data'] = train_data['raw_data'].astype('string')
train_data['id'] = train_data['id'].astype('int32')

train_data = train_data.drop(['image', "raw_data"], axis=1)

train_data.dtypes

id                      int32
parsed_data    string[python]
dtype: object

In [13]:
all_dict_records = [get_dict_from(train_data.iloc[0].iloc[1])]

for idx, row in tqdm(train_data.iterrows(), total=len(train_data), desc="Processing Rows", leave=True):
    dict_ = get_dict_from(row.iloc[1])
    for record_ in all_dict_records:
        if compare_items(dict_, record_):
            break
    else:
        all_dict_records.append(dict_)

Processing Rows: 100%|█████████████████████████████████| 2043/2043 [00:03<00:00, 576.66it/s]


In [14]:
len(all_dict_records)

24

In [10]:
train_data[train_data['id'].duplicated(keep=False)].sort_values(by=['id'])

,id,parsed_data,raw_data
0,0,"{""xml"": """", ""json"": ""{'header': {'invoice_no':...","{""ocr_words"": ""['Invoice no: 40378170', 'Date ..."
425,0,"{""xml"": ""<s_receipt><s_total></s_total><s_tips...","{""ocr_words"": ""['SAFEWAY', 'STORE MGR TOM MILA..."
1,1,"{""xml"": """", ""json"": ""{'header': {'invoice_no':...","{""ocr_words"": ""['Invoice no: 61356291', 'Date ..."
426,1,"{""xml"": ""<s_receipt><s_total>$89.09 $89.09</s_...","{""ocr_words"": ""[\""Walmart'\"", 'S', 'SUPERCENTE..."
112,2,"{""xml"": """", ""json"": ""{'header': {'invoice_no':...","{""ocr_words"": ""['Invoice no: 49565075', 'Date ..."
...,...,...,...
1403,422,"{""xml"": ""<s_receipt><s_total>2.10</s_total><s_...","{""ocr_words"": ""['Starbucks Coffee', 'Raleigh M..."
1404,423,"{""xml"": ""<s_receipt><s_total>61.33</s_total><s...","{""ocr_words"": ""['PORTUGAL WINE BAR & GRILL', '..."
361,423,"{""xml"": """", ""json"": ""{'header': {'invoice_no':...","{""ocr_words"": ""['Invoice no: 66430808', 'Date ..."
362,424,"{""xml"": """", ""json"": ""{'header': {'invoice_no':...","{""ocr_words"": ""['Invoice no: 94914402', 'Date ..."


In [47]:
train_data[train_data['parsed_data'].str.startswith('{"xml": "<', na='False')]

,id,parsed_data
425,0,"{""xml"": ""<s_receipt><s_total></s_total><s_tips..."
426,1,"{""xml"": ""<s_receipt><s_total>$89.09 $89.09</s_..."
427,10,"{""xml"": ""<s_receipt><s_total>Total:82.20</s_to..."
428,100,"{""xml"": ""<s_receipt><s_total>131.08 131.08</s_..."
429,1000,"{""xml"": ""<s_receipt><s_total>48.65 48.65EUR</s..."
...,...,...
2038,995,"{""xml"": ""<s_receipt><s_total>\u00a34.18</s_tot..."
2039,996,"{""xml"": ""<s_receipt><s_total>39.13</s_total><s..."
2040,997,"{""xml"": ""<s_receipt><s_total>26.91</s_total><s..."
2041,998,"{""xml"": ""<s_receipt><s_total>$10.23</s_total><..."


In [61]:
get_dict_from(train_data.iloc[1404].iloc[1])

{'store_name': 'PORTUGALWINEBAR&GRILL',
 'store_addr': 'PalmCoast,FL32137 15PalmHarborVillageWay#A',
 'telephone': '(386)447-3872',
 'date': '11/27/2016 11/27/2016',
 'time': '16:02:36',
 'subtotal': '57.30',
 'tax': '4.03',
 'total': '61.33',
 'ignore': '',
 'tips': '$8.59 $10.31 $11.46',
 'line_items': [{'item_key': '',
   'item_name': 'EXPRESSO',
   'item_value': '@1.50',
   'item_quantity': '1'},
  {'item_key': '',
   'item_name': 'FLAN',
   'item_value': '@5.00',
   'item_quantity': '1'},
  {'item_key': '',
   'item_name': 'SERRADURA',
   'item_value': '@4.95',
   'item_quantity': '1'},
  {'item_key': '',
   'item_name': 'CODFISH(BRAS)STYLE',
   'item_value': '@14.95',
   'item_quantity': '1'},
  {'item_key': '',
   'item_name': 'CALAMARI(ARROZ)RICESTE',
   'item_value': '@15.95',
   'item_quantity': '1'},
  {'item_key': '',
   'item_name': 'GRILLCHICKENBREAST',
   'item_value': '@14.95',
   'item_quantity': '1'}]}

In [62]:
get_dict_from(train_data.iloc[361].iloc[1])

{'header': {'invoice_no': '66430808',
  'invoice_date': '06/18/2020',
  'seller': 'Rowe-Lee 0226 Meghan Ranch North Codyborough, NM 91217',
  'client': 'Acosta, Price and West 8075 White Stream South Shawnburgh, NC 04517',
  'seller_tax_id': '952-73-6413',
  'client_tax_id': '951-94-4314',
  'iban': 'GB45FFLW23217428528260'},
 'items': [{'item_desc': 'Diamond Cut Crystal Whisky Glass Coketail Glasses Wine Glass 310ml Set of 6',
   'item_qty': '5.00',
   'item_net_price': '26.00',
   'item_net_worth': '130.00',
   'item_vat': '10%',
   'item_gross_worth': '143.00'},
  {'item_desc': 'Hicoup Waiters Corkscrew Wine Bottle Opener Professional Stainless Steel Black',
   'item_qty': '1.00',
   'item_net_price': '16.97',
   'item_net_worth': '16.97',
   'item_vat': '10%',
   'item_gross_worth': '18.67'},
  {'item_desc': 'EU Blichmann Riptide Brewing Pump Hombrew Beer Wine Food Grade',
   'item_qty': '2.00',
   'item_net_price': '200.00',
   'item_net_worth': '400.00',
   'item_vat': '10%',
   

In [42]:
train_data[train_data['id'] == 1617]


,id,parsed_data
1113,1617,"{""xml"": ""<s_receipt><s_total>$58.73</s_total><..."


In [9]:
1617+425

2042

In [27]:
for i in range(100):
    print(len())

TypeError: unhashable type: 'Series'

In [7]:
train_data['id'].nunique()

1618